# Brain Tumor Classification Task
Alessio Carnevale, Manuel Cattoni, Carlo Schillaci

Dataset link: [Kaggle](https://www.kaggle.com/datasets/pkdarabi/brain-tumor-image-dataset-semantic-segmentation/data)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import json

In [7]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("pkdarabi/brain-tumor-image-dataset-semantic-segmentation")

print("Path to dataset files:", path)

Path to dataset files: /Users/alessiocarnevale/.cache/kagglehub/datasets/pkdarabi/brain-tumor-image-dataset-semantic-segmentation/versions/1


In [14]:
with open('Dataset/train/_annotations.coco.json','r') as file:
    data = json.load(file)

for img in data['images'][:10]:
    print(img['file_name'])
    print(img)

2256_jpg.rf.3afd7903eaf3f3c5aa8da4bbb928bc19.jpg
{'id': 0, 'license': 1, 'file_name': '2256_jpg.rf.3afd7903eaf3f3c5aa8da4bbb928bc19.jpg', 'height': 640, 'width': 640, 'date_captured': '2023-08-19T04:37:54+00:00'}
2871_jpg.rf.3b6eadfbb369abc2b3bcb52b406b74f2.jpg
{'id': 1, 'license': 1, 'file_name': '2871_jpg.rf.3b6eadfbb369abc2b3bcb52b406b74f2.jpg', 'height': 640, 'width': 640, 'date_captured': '2023-08-19T04:37:54+00:00'}
2921_jpg.rf.3b952f91f27a6248091e7601c22323ad.jpg
{'id': 2, 'license': 1, 'file_name': '2921_jpg.rf.3b952f91f27a6248091e7601c22323ad.jpg', 'height': 640, 'width': 640, 'date_captured': '2023-08-19T04:37:54+00:00'}
2777_jpg.rf.3b60ea7f7e70552e70e41528052018bd.jpg
{'id': 3, 'license': 1, 'file_name': '2777_jpg.rf.3b60ea7f7e70552e70e41528052018bd.jpg', 'height': 640, 'width': 640, 'date_captured': '2023-08-19T04:37:54+00:00'}
2860_jpg.rf.3bb87fa4f938af5abfb1e17676ec1dad.jpg
{'id': 4, 'license': 1, 'file_name': '2860_jpg.rf.3bb87fa4f938af5abfb1e17676ec1dad.jpg', 'height': 